In [5]:
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from txtai.embeddings import Embeddings
from fuzzywuzzy import fuzz
from difflib import SequenceMatcher
import os

import torch
import numpy as np
import cv2
import re
import matplotlib.pyplot as plt
from diffusers import AutoPipelineForText2Image


RuntimeError: Failed to import diffusers.pipelines.auto_pipeline because of the following error (look up to see its traceback):
No module named 'triton.ops'

In [3]:
pip install -U "peft>=0.17.0" accelerate transformers safetensors


Defaulting to user installation because normal site-packages is not writeable
  Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (485 kB)
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.5.3
    Uninstalling safetensors-0.5.3:
      Successfully uninstalled safetensors-0.5.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.1
    Uninstalling accelerate-0.30.1:
      Successfully uninstalled accelerate-0.30.1
  Attempting uninstall: peft━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [accelerate]
    Found existing installation: peft 0.11.1m╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [accelerate]
    Uninstalling peft-0.11.1:━╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [accelerate]
      Successfully uninstalled peft-0.11.137m╺━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/3 [accelerate]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Load model in 16-bit precision
pipeline = AutoPipelineForText2Image.from_pretrained(
    'black-forest-labs/FLUX.1-schnell', torch_dtype=torch.bfloat16
).to("cuda")

# Load LoRA weights
pipeline.load_lora_weights(
    'output_with_90_percent_images/my_first_flux_lora_v1', 
    weight_name='my_first_flux_lora_v1_000070000.safetensors'
)


In [ ]:
import os
import gradio as gr

# Set Gradio temp directory to a writable path
os.environ["GRADIO_TEMP_DIR"] = "/home/asadullah_yousaf/gradio_cache"

# Ensure the directory exists
os.makedirs(os.environ["GRADIO_TEMP_DIR"], exist_ok=True)

In [4]:
import gradio as gr

# Function to generate images
def generate_image(prompt):
    
    prompt = prompt
    # Initialize txtai embeddings
    embeddings = Embeddings({"path": "sentence-transformers/all-MiniLM-L6-v2"})
    model_path = "models/txtai_prompt_index"

    # Load the model (for future use)
    embeddings.load(model_path)

    # Search for top 3 similar prompts
    results = embeddings.search(prompt, 3)

    # Function to calculate word-level differences
    def get_differences(text1, text2):
        matcher = SequenceMatcher(None, text1.split(), text2.split())
        diffs = [text2[b1:b2] for tag, a1, a2, b1, b2 in matcher.get_opcodes() if tag != 'equal']
        return " ".join(diffs)

    # Load dataset
    df = pd.read_csv("dataset_with_80000_with_concise_prompt.csv")
    data = df["text"]
    
    # Process results with threshold
    THRESHOLD = 0.85  # 95% similarity threshold
    valid_images = []

    for idx, similarity in results:
        existing_text = data[idx]

        # Calculate fuzzy similarity
        fuzzy_similarity = fuzz.ratio(prompt, existing_text) / 100.0

        # Weighted similarity score
        final_similarity = (similarity * 0.7) + (fuzzy_similarity * 0.3)

        # Ignore images below 95% similarity
        if final_similarity < THRESHOLD:
            continue

        # Get minor differences
        differences = get_differences(prompt, existing_text)

        print(f"🔹 **Existing Prompt:**\n{existing_text}\n")
        print(f"✅ **Semantic Similarity:** {similarity:.2f}")
        print(f"✅ **Fuzzy Similarity:** {fuzzy_similarity:.2f}")
        print(f"✅ **Final Score:** {final_similarity:.2f}")
        print(f"⚠️ **Minor Differences:** {differences}\n{'-'*80}\n")

        valid_images.append((idx, final_similarity))

    # Display only valid images
    if valid_images:
        fig, axes = plt.subplots(1, len(valid_images), figsize=(5 * len(valid_images), 5))

        if len(valid_images) == 1:
            axes = [axes]  # Make single image case work

        for i, (img_idx, confidence) in enumerate(valid_images):
            img_path = df["Image_Path"][img_idx].replace("floorplan_image", "floorplan_image")
            img = Image.open(img_path)

            axes[i].imshow(img)
            axes[i].axis("off")
            axes[i].set_title(f"Image {i+1}\nConfidence: {confidence * 100:.2f}%")

        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ No images met the 95% similarity threshold!")
    
    
    # Create an array to store images
    image_array = []

    if valid_images:
        for img_idx, _ in valid_images:  # Ignore confidence score
            img_path = df["Image_Path"][img_idx].replace("floorplan_image", "floorplan_image")
            img = Image.open(img_path)
            image_array.append(img)  # Store only the image

    # Now `image_array` contains only the images


    def generate_images(prompt, num_images=10):
        generated_images = []

        for i in range(num_images):
            image = pipeline(prompt).images[0]  # Generate image
            image_path = f"generated_image_{i+1}.png"
            image.save(image_path)  # Save the image
            generated_images.append(image_path)  # Store path

            # Display image
            plt.figure(figsize=(4, 4))
            plt.imshow(image)
            plt.axis('off')
            plt.title(f'Generated Image {i+1}')
            plt.show()

        return generated_images
    # Define color mappings for different rooms
    AREA_COLOR_MAP = {
        (238, 232, 170): "Living Room",    # Light Yellow
        (254, 215, 0): "Common Room",      # Yellow
        (240, 128, 127): "Kitchen",        # Light Coral
        (173, 217, 230): "Bathroom",       # Light Blue
        (254, 165, 0): "Master Room",      # Orange
        (106, 142, 34): "Balcony",         # Olive Green
        (218, 112, 213): "Storage Room",   # Orchid
    }

    def count_large_rooms(image, color_map, tolerance=0, min_pixels=500):
        """Count connected components with pixel area > min_pixels."""
        room_counts = {label: 0 for label in color_map.values()}  

        for color, label in color_map.items():
            lower_bound = np.array([max(0, c - tolerance) for c in color], dtype=np.uint8)
            upper_bound = np.array([min(255, c + tolerance) for c in color], dtype=np.uint8)

            # Create a mask for the current color
            mask = cv2.inRange(image, lower_bound, upper_bound)

            # Find connected components
            num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)

            # Count only components with area > min_pixels (excluding background)
            room_counts[label] = sum(1 for i in range(1, num_labels) if stats[i, cv2.CC_STAT_AREA] > min_pixels)

        return room_counts
    def extract_room_counts_from_prompt(prompt):
        """Extract expected room counts from the text prompt."""
        normalized_names = {room.lower(): room for room in AREA_COLOR_MAP.values()}
        room_counts = {room: 0 for room in AREA_COLOR_MAP.values()}

        room_pattern = r"(balcony|bathroom|living room|master room|kitchen|common room|storage room)"
        matches = re.findall(room_pattern, prompt, re.IGNORECASE)

        for match in matches:
            room_type = normalized_names.get(match.lower(), None)
            if room_type:
                room_counts[room_type] += 1

        return room_counts

    def calculate_loss(image_counts, prompt_counts):
        """Calculate the absolute difference between expected and actual room counts."""
        total_loss = sum(abs(image_counts[room] - prompt_counts[room]) for room in image_counts)

        room_differences = {
            room: {
                "Image Count": image_counts[room],
                "Prompt Count": prompt_counts[room],
                "Difference": abs(image_counts[room] - prompt_counts[room])
            }
            for room in image_counts
        }

        return total_loss, room_differences

    # Global list to store losses
    image_losses = []

    def process_floor_plans(image_paths, prompt, tolerance=23, min_pixels=500):
        """Process floor plans, compare with prompt, and calculate loss."""

        global image_losses  # Use global variable to store losses
        image_losses = []  # Reset for each new batch of images

        prompt_room_counts = extract_room_counts_from_prompt(prompt)

        plt.figure(figsize=(12, 4))
        room_differences_per_image = []  # Store room count differences per image

        for i, image_path in enumerate(image_paths):
            # Load image
            original_image = cv2.imread(image_path)
            if original_image is None:
                print(f"Error: Image not found at {image_path}")
                continue

            # Convert BGR to RGB
            original_image_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)

            # Get room counts
            image_room_counts = count_large_rooms(original_image_rgb, AREA_COLOR_MAP, tolerance, min_pixels)

            # Compute loss and detailed differences
            loss, room_differences = calculate_loss(image_room_counts, prompt_room_counts)
            image_losses.append((image_path, loss))  # Store image name & loss

            room_differences_per_image.append(room_differences)

            # Display image with loss
            plt.subplot(1, len(image_paths), i + 1)
            plt.imshow(original_image_rgb)
            plt.axis("off")
            plt.title(f"Loss = {loss}")


        # Print loss and room count differences for each image
        for image_path, room_differences in zip(image_paths, room_differences_per_image):
            print(f"\nImage: {image_path}")
            for room, details in room_differences.items():
                print(f"{room}: Image count = {details['Image Count']}, "
                      f"Prompt count = {details['Prompt Count']}, "
                      f"Difference = {details['Difference']}")
        return image_losses

    # prompt = '1 balcony 50 sqft ratio 13:3 green, master room 200 sqft ratio 7:11 yellow, bathroom 100 sqft ratio 1:1 blue, common room 150 sqft ratio 15:16 orange, 2 balcony 23 sqft ratio 13:9 green, living room 800 sqft ratio 11:9 light yellow.'

    # Generate 3 images
    generated_images = generate_images(prompt, num_images=10)

    # Process generated images
    image_losses = process_floor_plans(generated_images, prompt)
    
    print("+++++++++++++++++++++++++++++++")
    print(image_losses)
    print("+++++++++++++++++++++++++++++++")
    

    global best_images_array
    best_images_array = []  # New array to store best images

    if image_losses:
        # Filter images with loss ≤ 2
        filtered_images = [(img_name, loss) for img_name, loss in image_losses if loss <= 2]

        if not filtered_images:
            print("No images found with loss ≤ 2.")
        else:
            # Sort images by loss (ascending order)
            filtered_images.sort(key=lambda x: x[1])

            # Select up to 3 images with the least loss
            best_images = filtered_images[:3]
            best_images_array = [img_name for img_name, _ in best_images]  # Store image names

            # Print loss for selected images
            print("\nSelected Images with Loss ≤ 2:")
            for img_name, loss in best_images:
                print(f"{img_name}: Loss = {loss}")

            # Display best images
            plt.figure(figsize=(12, 4))  # Adjust figure size for 3 images
            for i, img_name in enumerate(best_images_array):
                img = cv2.imread(img_name)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                plt.subplot(1, 3, i + 1)  # Adjust subplot for 3 images
                plt.imshow(img)
                plt.axis("off")
                plt.title(f"Best Image {i+1}")

    else:
        print("No images processed yet.")




    best_images_array.extend(image_array)
    # Convert file paths to PIL images
    best_images_array = [Image.open(img) if isinstance(img, str) else img for img in best_images_array]

    
    return best_images_array



In [5]:
# Create Gradio API
app = gr.Interface(fn=generate_image, inputs="text", outputs="gallery")

# Run the API on port 7860
app.launch(server_name="0.0.0.0", server_port=7881, share=True)

* Running on local URL:  http://0.0.0.0:7881
* Running on public URL: https://3d466a1d61659d9ffd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔹 **Existing Prompt:**
master room 250 sqft ratio 8:9 yellow, bathroom 50 sqft ratio 3:2 blue, kitchen 50 sqft ratio 3:2 coral red, common room 50 sqft ratio 11:8 orange, living room 550 sqft ratio 16:15 light yellow, balcony 100 sqft ratio 13:6 green.

✅ **Semantic Similarity:** 0.99
✅ **Fuzzy Similarity:** 0.70
✅ **Final Score:** 0.90
⚠️ **Minor Differences:** r o   50 sqft ratio  9 e o ba
--------------------------------------------------------------------------------

🔹 **Existing Prompt:**
master room 250 sqft ratio 10:11 yellow, bathroom 50 sqft ratio 16:13 blue, common room 150 sqft ratio 3:4 orange, kitchen 100 sqft ratio 14:11 coral red, living room 600 sqft ratio 3:4 light yellow, balcony 50 sqft ratio 7:2 green.

✅ **Semantic Similarity:** 0.99
✅ **Fuzzy Similarity:** 0.77
✅ **Final Score:** 0.92
⚠️ **Minor Differences:** r o    ft ratio 10:11 yellow, 
--------------------------------------------------------------------------------

🔹 **Existing Prompt:**
master room 250 sqf

/tmp/ipykernel_118242/240800156.py:71: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


  0%|          | 0/28 [00:00<?, ?it/s]

/tmp/ipykernel_118242/240800156.py:102: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


...

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]

  0%|          | 0/28 [00:00<?, ?it/s]


Image: generated_image_1.png
Living Room: Image count = 2, Prompt count = 1, Difference = 1
Common Room: Image count = 1, Prompt count = 1, Difference = 0
Kitchen: Image count = 1, Prompt count = 1, Difference = 0
Bathroom: Image count = 1, Prompt count = 1, Difference = 0
Master Room: Image count = 1, Prompt count = 1, Difference = 0
Balcony: Image count = 1, Prompt count = 1, Difference = 0
Storage Room: Image count = 0, Prompt count = 0, Difference = 0

Image: generated_image_2.png
Living Room: Image count = 1, Prompt count = 1, Difference = 0
Common Room: Image count = 1, Prompt count = 1, Difference = 0
Kitchen: Image count = 1, Prompt count = 1, Difference = 0
Bathroom: Image count = 2, Prompt count = 1, Difference = 1
Master Room: Image count = 2, Prompt count = 1, Difference = 1
Balcony: Image count = 1, Prompt count = 1, Difference = 0
Storage Room: Image count = 0, Prompt count = 0, Difference = 0

Image: generated_image_3.png
Living Room: Image count = 1, Prompt count = 1, 